In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import zipfile
import os

In [ ]:
# ==================== MODEL ARCHITECTURE ====================

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(channels, channels, 3),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(channels, channels, 3),
            nn.InstanceNorm2d(channels)
        )
    
    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=64, n_residual=9):
        super().__init__()
        
        self.initial = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, features, 7),
            nn.InstanceNorm2d(features),
            nn.ReLU(inplace=True)
        )
        
        self.down_blocks = nn.ModuleList()
        curr_dim = features
        for _ in range(2):
            self.down_blocks.append(nn.Sequential(
                nn.Conv2d(curr_dim, curr_dim * 2, 3, stride=2, padding=1),
                nn.InstanceNorm2d(curr_dim * 2),
                nn.ReLU(inplace=True)
            ))
            curr_dim *= 2
        
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(curr_dim) for _ in range(n_residual)]
        )
        
        self.up_blocks = nn.ModuleList()
        for _ in range(2):
            self.up_blocks.append(nn.Sequential(
                nn.ConvTranspose2d(curr_dim, curr_dim // 2, 3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(curr_dim // 2),
                nn.ReLU(inplace=True)
            ))
            curr_dim //= 2
        
        self.output = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(curr_dim, out_channels, 7),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = self.initial(x)
        for down in self.down_blocks:
            x = down(x)
        x = self.res_blocks(x)
        for up in self.up_blocks:
            x = up(x)
        return self.output(x)


# ==================== CONFIGURATION ====================

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

CHECKPOINT_PATH = '/kaggle/input/cyclegan-lsgan-checkpoint/lsgan_epoch_29.pth'
PHOTO_DIR = '/kaggle/input/gan-getting-started/photo_jpg'
OUTPUT_ZIP = '/kaggle/working/images.zip'


# ==================== LOAD MODEL ====================

print("Loading trained model...")
G_P2M = Generator().to(DEVICE)

checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
G_P2M.load_state_dict(checkpoint['G_P2M_state'])
G_P2M.eval()
print("Model loaded successfully!")

# ==================== SETUP TRANSFORM ====================

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# ==================== LOAD TEST PHOTOS ====================

print("\nLoading test photos...")
test_photos = sorted(list(Path(PHOTO_DIR).glob('*.jpg')))
print(f"Found {len(test_photos)} test photos")

# ==================== GENERATE IMAGES ====================

print("\nGenerating Monet-style images...")
print(f"Output: {OUTPUT_ZIP}")

with zipfile.ZipFile(OUTPUT_ZIP, 'w', zipfile.ZIP_DEFLATED) as zipf:
    with torch.no_grad():
        for photo_path in tqdm(test_photos):
            photo = Image.open(photo_path).convert('RGB')
            photo_tensor = transform(photo).unsqueeze(0).to(DEVICE)
            
            fake_monet = G_P2M(photo_tensor)
            
            fake_monet = (fake_monet * 0.5 + 0.5).clamp(0, 1)
            fake_monet = fake_monet.squeeze(0).cpu()
            
            fake_monet_pil = transforms.ToPILImage()(fake_monet)
            
            from io import BytesIO
            img_buffer = BytesIO()
            fake_monet_pil.save(img_buffer, format='JPEG', quality=95)
            img_buffer.seek(0)
            
            zipf.writestr(photo_path.name, img_buffer.read())


# ==================== VERIFY OUTPUT ====================

print("\nVerifying submission...")
with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zipf:
    num_images = len(zipf.namelist())
    print(f"Generated {num_images} images")
    print(f"Zip size: {os.path.getsize(OUTPUT_ZIP) / (1024*1024):.1f} MB")
    
    print(f"\nSample filenames:")
    for name in zipf.namelist()[:5]:
        print(f"  - {name}")

print("\n" + "="*60)
print("SUBMISSION READY!")
print("="*60)
print(f"\nOutput file: {OUTPUT_ZIP}")